In [1]:
%%time
import os
from os.path import join
from concurrent.futures import ThreadPoolExecutor

base_path = '/work/mflora/wofs-cast-data/datasets_2hr'
years = ['2019']

def get_files_for_year(year):
    year_path = join(base_path, year)
    with os.scandir(year_path) as it:
        return [join(year_path, entry.name) for entry in it if entry.is_file()]

with ThreadPoolExecutor() as executor:
    paths = []
    for files in executor.map(get_files_for_year, years):
        paths.extend(files)

print(len(paths))

5130
CPU times: user 4.87 ms, sys: 2.25 ms, total: 7.12 ms
Wall time: 6.98 ms


In [2]:
%%time 
import xarray as xr
import zarr
import dask 
import os

def netcdf_to_zarr(netcdf_path, compressor=None, chunk_sizes=None):
    """
    Convert a NetCDF file to Zarr format optimized for I/O speeds.

    Args:
        netcdf_path (str): Path to the input NetCDF file.
        compressor (zarr.Compressor, optional): Zarr compressor to use. Defaults to None.
        chunk_sizes (dict, optional): Dictionary specifying chunk sizes. Defaults to None.

    Returns:
        None
    """
    zarr_path = netcdf_path.replace('datasets', 'datasets_zarr').replace('.nc', '.zarr')
    
    # Open the NetCDF file
    ds = xr.open_dataset(netcdf_path, chunks=chunk_sizes)

    # If no compressor is specified, use the default compressor
    if compressor is None:
        compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=zarr.Blosc.SHUFFLE)

    # Set encoding for each variable to use the specified compressor
    encoding = {var: {'compressor': compressor} for var in ds.data_vars}

    # Write the dataset to Zarr format
    # Ensure output directory exists
    if not os.path.exists(os.path.dirname(zarr_path)):
        os.makedirs(os.path.dirname(zarr_path), exist_ok=True)
    
    ds.to_zarr(zarr_path, mode='w', encoding=encoding, consolidated=True)
    ds.close()
    
    print(f'Saved {zarr_path}...')
    
    return 'Done'


results = dask.compute(*[dask.delayed(netcdf_to_zarr)(u, chunk_sizes={}) for u in paths])

In [2]:
paths[0]

'/work/mflora/wofs-cast-data/datasets_2hr/2019/wrfwof_2019-06-22_010000_to_2019-06-22_031000__10min__ens_mem_17.nc'

In [2]:
%%time
import xarray as xr

ds = xr.open_dataset(paths[0], chunks={})#{'time' : 1, 'datetime' : 1})

CPU times: user 1.68 s, sys: 10.6 s, total: 12.3 s
Wall time: 523 ms


In [8]:
ds['T']

<xarray.DataArray 'T' (time: 14, level: 17, lat: 150, lon: 150)>
dask.array<open_dataset-T, shape=(14, 17, 150, 150), dtype=float32, chunksize=(1, 17, 150, 150), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.12 32.15 32.17 32.2 ... 36.11 36.14 36.17 36.19
  * lon      (lon) float32 79.53 79.56 79.59 79.62 ... 84.09 84.12 84.16 84.19
  * time     (time) timedelta64[ns] 00:00:00 00:10:00 ... 02:00:00 02:10:00
  * level    (level) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
Attributes:
    FieldType:                                          104
    MemoryOrder:                                        XYZ
    description:                                        perturbation potentia...
    units:                                              K
    stagger:                                            
    coordinates:                                        XLONG XLAT XTIME
    QuantizeGranularBitRoundNumberOfSignificantDigits:  3